In [1]:
import pandas as pd
import numpy as np

#re and string will be used to process the data
import re 
import string

#Import all the dependencies
import os
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import gensim

#import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from tensorboard.plugins import projector

#Just to get rid of warnings
import warnings
warnings.filterwarnings("ignore")

listingsDF = pd.read_csv('listings.csv')

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:

#Instead of dropping we select what we need
listingsDF = listingsDF[['id', 'name','description', 
                         'host_id', 'host_name', 'property_type', 'price', 
                         'number_of_reviews', 'review_scores_rating']]

#We drop all rows with empty cells
listingsDF = listingsDF.dropna(axis=0, how='any')

#Remove the '$' from price
listingsDF.price = listingsDF.price.str.replace('[$]', '')

#Remove the ',' from price
listingsDF.price = listingsDF.price.str.replace(',', '')

#Convert price from object to float
listingsDF['price'] = listingsDF['price'].astype(float)

#listingsDF.head()
listingsDF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3171 entries, 0 to 3814
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    3171 non-null   int64  
 1   name                  3171 non-null   object 
 2   description           3171 non-null   object 
 3   host_id               3171 non-null   int64  
 4   host_name             3171 non-null   object 
 5   property_type         3171 non-null   object 
 6   price                 3171 non-null   float64
 7   number_of_reviews     3171 non-null   int64  
 8   review_scores_rating  3171 non-null   float64
dtypes: float64(2), int64(3), object(4)
memory usage: 247.7+ KB


In [3]:
aDF = listingsDF[['id','name','description']]

In [4]:
aDF['full_description'] = aDF['description']

# Data Pre-processing

In [5]:
pd.set_option('display.max_colwidth', -1)

def clean(text):
    text = text.lower()
    text = re.sub('[^\w\s]', '', text) #This removes all the punctuations
    text = re.sub(r'\n',' ', text) #This relaces the \n with space
    text = re.sub(r'\r','', text) #\r
    text = re.sub('[^0-9a-z #+_]', '', text) #Special chars
    
    return text

aDF['description'] = aDF['description'].apply(clean)

In [6]:
aDF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3171 entries, 0 to 3814
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                3171 non-null   int64 
 1   name              3171 non-null   object
 2   description       3171 non-null   object
 3   full_description  3171 non-null   object
dtypes: int64(1), object(3)
memory usage: 123.9+ KB


In [7]:
aDF['id'] = aDF['id'].astype(str)

In [8]:
desclist = list(aDF['description'])

In [9]:
name_label = list(aDF['name']) #Keep this for tagging 

# Convert the listing_id to be used as tagging for d2v

In [10]:
id_label = list(aDF['id'])
id_label.append('0')
len(id_label)

3172

 # Prompt user input here

In [11]:
u_input = 'I want a huge huge room and a  quiet area which is near the shopping malls!!'
user_input_label = 'user_input'

cleaned_input = clean(u_input)
inputtoken = word_tokenize(u_input)
desclist.append(u_input)
name_label.append('user_input')

In [12]:
len(desclist)

3172

In [13]:
aDF = aDF.append({'id':0, 'name':user_input_label, 'description':cleaned_input, 'full_description':u_input}, ignore_index=True)
aDF

,id,name,description,full_description
0,241032,Stylish Queen Anne Apartment,make your self at home in this charming onebedroom apartment centrallylocated on the west side of queen anne hill this elegantlydecorated completely private apartment bottom unit of a duplex has an open floor plan bamboo floors a fully equipped kitchen a tv dvd player basic cable and a very cozy bedroom with a queensize bed the unit sleeps up to four two in the bedroom and two on the very comfortable fold out couch linens included and includes free wifi and laundry the apartment opens onto a private deck complete with its own bbq overlooking a garden and a forest of black bamboo the apartment is perfectlylocated just one block from the bus lines where you can catch a bus and be downtown seattle in fifteen minutes or historic ballard in ten or a quick fiveminute walk will bring you to whole foods and peets coffee or take a fifteen minute walk to the top of queen anne hill where you will find a variety of eclectic shops bars and restaurants there is no,"Make your self at home in this charming one-bedroom apartment, centrally-located on the west side of Queen Anne hill. This elegantly-decorated, completely private apartment (bottom unit of a duplex) has an open floor plan, bamboo floors, a fully equipped kitchen, a TV, DVD player, basic cable, and a very cozy bedroom with a queen-size bed. The unit sleeps up to four (two in the bedroom and two on the very comfortable fold out couch, linens included) and includes free WiFi and laundry. The apartment opens onto a private deck, complete with it's own BBQ, overlooking a garden and a forest of black bamboo. The Apartment is perfectly-located just one block from the bus lines where you can catch a bus and be downtown Seattle in fifteen minutes or historic Ballard in ten or a quick five-minute walk will bring you to Whole Foods and Peet's Coffee or take a fifteen minute walk to the top of Queen Anne Hill where you will find a variety of eclectic shops, bars, and restaurants. There is no"
1,953595,Bright & Airy Queen Anne Apartment,chemically sensitive weve removed the irritants triggering allergy or asthma attacks like carpeting forced air used pillows all culprits that harbor fungus mold bacteria no smoking no pets designed for healthy living so breathe easy beautiful hypoallergenic apartment in an extremely safe quiet and pedestrianfriendly section of queen anne a leafygreen location that puts the best of seattle at your doorstep free wifi free parking even free pillows whats special about this place hypoallergenic bedding mattressescovers and new pillows with each rental so feel free to take your new pillows with you a beautiful restoration mixing period details modern sensibilities hardwood floors throughout white subway tile lowno vocs nontoxic paints keep this homeawayfromhome clean smart and healthy radiant heat and oil radiators mean no blowing hot air or dust great amenities free wifi internet tv led flatscreen fullyequipped kitchen with st,"Chemically sensitive? We've removed the irritants triggering allergy or asthma attacks, like carpeting, forced air & used pillows, all culprits that harbor fungus, mold & bacteria. No smoking, no pets. Designed for healthy living, so breathe easy. Beautiful, hypoallergenic apartment in an extremely safe, quiet and pedestrian-friendly section of Queen Anne. A leafy-green location that puts the best of Seattle at your doorstep. Free WiFi, free parking, ...even free pillows! What's special about this place? Hypo-allergenic bedding, mattresses/covers and new pillows with each rental. (So feel free to take your new pillows with you!) A beautiful restoration mixing period details & modern sensibilities: Hardwood floors throughout, white subway tile, low/no VOCs & non-toxic paints keep this home-away-from-home clean, smart and healthy. Radiant heat and oil radiators mean no blowing hot air or dust. Great Amenities: Free WiFi & internet TV (LED flatscreen). Fully-equipped k

# Tag first

In [14]:
train_corpus = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[id_label[i]]) for i, _d in enumerate(desclist)]

In [15]:
# tags=[str(1)]
# tags
# tags=[name_label[i]]

In [16]:
train_corpus

[TaggedDocument(words=['make', 'your', 'self', 'at', 'home', 'in', 'this', 'charming', 'onebedroom', 'apartment', 'centrallylocated', 'on', 'the', 'west', 'side', 'of', 'queen', 'anne', 'hill', 'this', 'elegantlydecorated', 'completely', 'private', 'apartment', 'bottom', 'unit', 'of', 'a', 'duplex', 'has', 'an', 'open', 'floor', 'plan', 'bamboo', 'floors', 'a', 'fully', 'equipped', 'kitchen', 'a', 'tv', 'dvd', 'player', 'basic', 'cable', 'and', 'a', 'very', 'cozy', 'bedroom', 'with', 'a', 'queensize', 'bed', 'the', 'unit', 'sleeps', 'up', 'to', 'four', 'two', 'in', 'the', 'bedroom', 'and', 'two', 'on', 'the', 'very', 'comfortable', 'fold', 'out', 'couch', 'linens', 'included', 'and', 'includes', 'free', 'wifi', 'and', 'laundry', 'the', 'apartment', 'opens', 'onto', 'a', 'private', 'deck', 'complete', 'with', 'its', 'own', 'bbq', 'overlooking', 'a', 'garden', 'and', 'a', 'forest', 'of', 'black', 'bamboo', 'the', 'apartment', 'is', 'perfectlylocated', 'just', 'one', 'block', 'from', 'the

# Start model training

In [17]:
model = Doc2Vec(vector_size=40, min_count=1, epochs=20, dm=1, hs=1, window=2)

In [18]:

model.build_vocab(train_corpus)

model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)
model.save("d2v.model")
print("Model Saved") #For visualization later

Model Saved


In [19]:
vector = model.infer_vector(inputtoken)
print(vector)

[-0.39121518 -0.24106032 -0.5413575  -0.6569513  -0.5758826   0.41271088
  0.01971445 -0.16901374 -1.5329075  -0.69488853 -1.4231799  -0.21840504
  0.12943824 -0.5265459  -0.43273738 -0.81532747 -0.2287286  -0.9529539
 -0.9176233   0.10875785  0.5744128  -0.74567175 -0.04035493 -0.43589938
 -0.10480198  0.67046803  0.14505412  1.0012847   0.20097941  0.7551421
  0.29749146 -0.37101746 -0.5381334   0.20202145  0.22541524 -0.13965002
  1.6127785   0.17369418  0.05639822  0.54991895]


# Put the similarity of listings in a list

In [20]:
for doc_id in range(len(train_corpus)):
    inferred_vector = vector
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))


In [21]:
simsL = [list(e) for e in sims] #Convert to List of List

# Put it in a Data Frame so it looks nicer

In [22]:
descDF = aDF[['id', 'name', 'full_description']]
simsDF = pd.DataFrame(simsL, columns=['id','scores'])
resultDF = pd.merge(descDF, simsDF, on='id')

In [23]:
resultDF = resultDF[['id', 'name','full_description','scores']]
resultDF = resultDF.sort_values(by=['scores'], ascending=False)
resultDF.head(5)

,id,name,full_description,scores
91,4732995,Urban Oasis in the Heart of Ballard,"This listing is located in the middle of Ballard, just a block or two away from popular shops, restaurants & breweries that make this area so special! Just a short bus ride or Uber cab ride from downtown with pick up just outside your door! This one bedroom apartment in the heart of Ballard is spacious and lovely! Sleeps up to three: one full bed and one large couch. You have your own balcony which overlooks some trees and the lot next door. They are doing some construction on the lot, however they do not work on weekends and don't start working in the morning until 8am or later. You'll have the apartment to yourself! You'll also have access to an amazing gym, rooftop including sunbathing area and grills, Scene Room with huge TV for residents, courtyard and Mingle Room. All of thees spaces have free WiFi. Ballard is one of the most special neighborhoods in Seattle. It has an amazing bar and restaurant scene and lots of cute shops. This property is brand new and located right in the mi",0.584827
96,6337492,Nice 1 bedroom apartment in Ballard,"Nice, clean and quite 1 bedroom apartment. Waking distance from ballard, the locks and golden gardens. The unit has big windows &has lots of space to kick up your feet & make yourself at home. Experience Seattle's more nature filled areas. EXPLORE! It is close to night life and nature. My unit has access to fire pits, outdoor areas and a gym Please be respectful.. I will not be home. Other than giving and returning keys. Golden gardens, down town Ballard night life, the ballard locks. Lots of places to eat, local farmers market on Sundays The 44 bus stops right in front of my apartment complex",0.569382
1354,8505421,WONDERFUL LOCATION,"This beautiful space could not be in a more convenient location! Sandwiched between Downtown and Capitol Hill, it is walkable to Pike Place Market, the ferry and light rail stations, both stadiums as well as plenty of restaurant/bar/retail spots!",0.539517
779,9516608,Modern 4BR/3.5BA Near Downtown,"Our beautiful modern home is centrally located in colorful Capitol Hill. Walk to restaurants, bars, shopping, theatre, parks, 3 groceries, 2 farmers markets. Family and kid friendly. Walk/bus/10 min Lyft/Uber to Downtown, Pike Place, museums & more. Our beautiful, spacious modern home is fully furnished, extremely comfortable and close to everything. Whether you prefer getting around on foot or via Uber/Lyft, bus or car, the central location can't be beat. The perfect vacation home for families or couples who want to explore the city, spend time together and recharge in comfort between excursions. Our large home spans three above-ground floors, with three bedrooms, two full bathrooms and a laundry room upstairs; living room, dining room, kitchen, family room with small TV, two gas fireplaces, and a half bath on the main floor; a fourth bedroom with queen bed, a full bathroom, and entertainment room with large screen TV, foosball table and lots of games on the lower level. Covered bac",0.527995
2970,6066570,Classic Beautiful Seattle Home,"Beautiful and historic updated Seattle home in very nice and quiet Capitol Hill neighborhood close to everything. Wonderfully appointed for a big group with 6 bedrooms, 3 1/2 baths. It was the home of a former Seattle mayor! A beautiful, stately, (PHONE NUMBER HIDDEN) home with tons of charm, but remodeled nicely. Room for a big group with great outdoor entertaining space. Fantastic neighborhood in Capitol Hill that is quiet, yet close to great restaurants, cafes, shops and parks. Near bus line and close to downtown and freeways. The home has 5 bedrooms and 3 1/2 baths. On the second floor, a master suite with king bed, private deck and fantastic bathroom with shower, tub and two sinks, a guest bedroom with queen bed plus an adjoining bedroom with a single bunk bed share a beautiful bathroom with tub and separate shower. On the third floor, a bedroom

# Visualize

In [24]:
model.save_word2vec_format('doc_tensor.w2v', doctag_vec=True, word_vec=False)  

In [25]:
%run word2vec2tensor -i doc_tensor.w2v -o desc

2020-04-09 08:24:02,044 - word2vec2tensor - INFO - running word2vec2tensor.py -i doc_tensor.w2v -o desc
2020-04-09 08:24:02,046 - utils_any2vec - INFO - loading projection weights from doc_tensor.w2v
2020-04-09 08:24:02,197 - utils_any2vec - INFO - loaded (3172, 40) matrix from doc_tensor.w2v
2020-04-09 08:24:02,303 - word2vec2tensor - INFO - 2D tensor file saved to desc_tensor.tsv
2020-04-09 08:24:02,303 - word2vec2tensor - INFO - Tensor metadata file saved to desc_metadata.tsv
2020-04-09 08:24:02,304 - word2vec2tensor - INFO - finished running word2vec2tensor.py


In [26]:
if not os.path.exists('descproj'):
    os.makedirs('descproj')

with open('descproj/desc_metadata.tsv','w', encoding="utf-8") as w: #This part is to export vec and metadata tsv
        
    w.write('name\tdescription\n')
    for i,j in zip(aDF.name, aDF.full_description):
        w.write("%s\t%s\n" % (i,j))

In [27]:
tf.reset_default_graph() #Everything else is to run the projector on local tensorboard
sess = tf.InteractiveSession()

In [28]:
embedding = model.docvecs.vectors_docs

X = tf.Variable([0.0], name='desc_embedding')
tf.global_variables_initializer().run()

place = tf.placeholder(tf.float32, shape=embedding.shape)
set_x = tf.assign(X, place, validate_shape=False)
sess.run(tf.global_variables_initializer())
sess.run(set_x, feed_dict={place: embedding})


array([[-0.2546505 , -0.7001308 ,  1.6198004 , ..., -0.9263273 ,
        -0.59056216,  0.875621  ],
       [-0.8958435 , -0.81846833,  1.6122129 , ..., -0.9360349 ,
        -0.29881445,  0.7920834 ],
       [-0.0405117 ,  0.37693885,  0.68487847, ..., -0.74780697,
         0.4337005 ,  0.00454941],
       ...,
       [ 0.61424696, -0.04394154,  1.1372819 , ..., -0.19462878,
        -0.21838912,  0.82284147],
       [-0.6536365 ,  0.27842826,  0.12640081, ..., -0.2593753 ,
         0.3820938 ,  0.7359862 ],
       [-0.12359543, -0.3340944 , -0.48623878, ...,  0.01541484,
        -0.03913025,  0.46082494]], dtype=float32)

In [29]:
saver = tf.train.Saver()

In [30]:
writer = tf.summary.FileWriter('descproj', sess.graph)

In [31]:
config = projector.ProjectorConfig()
embed= config.embeddings.add()

In [32]:
embed.tensor_name = 'desc_embedding'
embed.metadata_path = 'desc_metadata.tsv'

In [33]:
projector.visualize_embeddings(writer, config)

saver.save(sess, 'descproj/model.ckpt')

'descproj/model.ckpt'

In [34]:
embedding.shape

(3172, 40)